In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering

In [3]:
file_path = 'dataset/n_ladrillera.csv'
df = pd.read_csv(file_path, quotechar='"', delimiter=";")
df['STOCK_ACTUAL'] = df['STOCK_ACTUAL'].str.replace(',', '.').astype(float)
df['TOTAL_CANTIDAD_MOVS'] = df['TOTAL_CANTIDAD_MOVS'].str.replace(',', '.').astype(float)
df['PMP'] = df['PMP'].str.replace(',', '.').astype(float)
df['TOTAL_IMPORTE_MOVS'] = df['TOTAL_IMPORTE_MOVS'].str.replace(',', '.').astype(float)
label_encoder = LabelEncoder()
df['ARTICULO'] = label_encoder.fit_transform(df['ARTICULO'])
df['TIPO_MATERIAL'] = label_encoder.fit_transform(df['TIPO_MATERIAL'])
df['CODIGO_FAMILIA'] = label_encoder.fit_transform(df['CODIGO_FAMILIA'])
df.drop(columns=['EMPRESA','PESO_NETO','MULTIPRESENTACION','ARTICULO','CODIGO_FAMILIA'], inplace=True)
columns_to_standardize = ['TIPO_MATERIAL','PMP','STOCK_ACTUAL','TOTAL_CANTIDAD_MOVS','TOTAL_IMPORTE_MOVS','TOTAL_MOVS']
scaler = StandardScaler()
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# IQR para detectar y eliminar outliers
columns = columns_to_standardize

filtered_df = df.copy()

for column in columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_df = filtered_df[(filtered_df[column] >= lower_bound) & (filtered_df[column] <= upper_bound)]


original_count = df.shape[0]
filtered_count = filtered_df.shape[0]
reduction = ((original_count - filtered_count) / original_count) * 100

print(f"El DataFrame se ha reducido en un {reduction}%")



El DataFrame se ha reducido en un 43.42105263157895%


In [ ]:
### PON AQUÍ TU CÓDIGO
# Calcula las correlaciones de Pearson entre todas las variables
correlation_matrix = df.corr(method='pearson')

# Configura el tamaño de la figura
plt.figure(figsize=(10, 8))

# Crea un mapa de calor de la matriz de correlaciones
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# Añade título
plt.title('Matriz de correlación de Pearson')

# Muestra el mapa de calor
plt.show()

In [ ]:
X = df.copy()

In [ ]:

# Aplicar Clustering Jerárquico
agg_model = AgglomerativeClustering(n_clusters=3)
y_agg = agg_model.fit_predict(X)

# Agregar etiquetas de cluster al dataframe
clusters_agg = pd.DataFrame(X, columns=X.columns)
clusters_agg['label'] = agg_model.labels_

color_map = {0: 'red', 1: 'green', 2: 'blue'}

# Visualización polar para Clustering Jerárquico
'''
polar_agg = clusters_agg.groupby("label").mean().reset_index()
polar_agg = pd.melt(polar_agg, id_vars=["label"])
fig_agg = px.line_polar(polar_agg, r="value", theta="variable", color="label", line_close=True, height=800, width=900, color_discrete_map=color_map)
fig_agg.show()
'''

# Visualización de distribución de clusters con Clustering Jerárquico
pie_agg = clusters_agg.groupby('label').size().reset_index()
pie_agg.columns = ['label', 'value']
fig_a = px.pie(pie_agg, values='value', names='label', color='label', height=400, width=400, color_discrete_map=color_map,
                 labels={"label": "Cluster", "value": "Número de Instancias"})
fig_a.show()

In [ ]:
def assign_colors_by_size(labels, color_order):
    cluster_sizes = pd.Series(labels).value_counts()
    sorted_clusters = cluster_sizes.index
    color_map = {sorted_clusters[i]: color_order[i] for i in range(len(sorted_clusters))}
    return labels.map(color_map)

color_order = ['0', '1', '2']

clusters_agg = pd.DataFrame(X, columns=X.columns)
clusters_agg['label'] = agg_model.labels_
clusters_agg['color_label'] = assign_colors_by_size(clusters_agg['label'], color_order)

pie_agg = clusters_agg.groupby('color_label').size().reset_index()
pie_agg.columns = ['color_label', 'value']
fig_a = px.pie(pie_agg, values='value', names='color_label', height=400, width=400, color='color_label',
                 labels={"color_label": "Cluster", "value": "Número de Instancias"})

fig_a.show()

In [ ]:
# Calcular métricas de evaluación
silhouette_avg = silhouette_score(X, y_agg)
davies_bouldin_avg = davies_bouldin_score(X, y_agg)

# Imprimir métricas
print("Silhouette Score:", silhouette_avg)
print("Davies-Bouldin Index:", davies_bouldin_avg)
print("")

In [ ]:
kmeansmodel = KMeans(n_clusters = 3, init='k-means++', random_state=0,n_init=10)
y_kmeans = kmeansmodel.fit_predict(X)

In [ ]:

clusters_f = pd.DataFrame(X, columns=X.columns)
clusters_f['label'] = kmeansmodel.labels_

color_map = {0: 'blue', 1: 'red', 2: 'green'}

# Visualización de distribución de clusters con Clustering Jerárquico
pie_k = clusters_f.groupby('label').size().reset_index()
pie_k.columns = ['label', 'value']
fig_k = px.pie(pie_k, values='value', names='label', color='label', height=400, width=400, color_discrete_map=color_map,
                 labels={"label": "Cluster", "value": "Número de Instancias"})

fig_k.show()

In [ ]:
# Calcular métricas de evaluación
silhouette_avg = silhouette_score(X, y_kmeans)
davies_bouldin_avg = davies_bouldin_score(X, y_kmeans)

# Imprimir métricas
print("Silhouette Score:", silhouette_avg)
print("Davies-Bouldin Index:", davies_bouldin_avg)
print("Innertia:",kmeansmodel.inertia_)

In [ ]:
def assign_colors_by_size(labels, color_order):
    cluster_sizes = pd.Series(labels).value_counts()
    sorted_clusters = cluster_sizes.index
    color_map = {sorted_clusters[i]: color_order[i] for i in range(len(sorted_clusters))}
    return labels.map(color_map)

color_order = ['blue', 'red', 'green']

clusters_kmeans = pd.DataFrame(X, columns=X.columns)
clusters_kmeans['label'] = kmeansmodel.labels_
clusters_kmeans['color_label'] = assign_colors_by_size(clusters_kmeans['label'], color_order)

pie_k = clusters_kmeans.groupby('color_label').size().reset_index()
pie_k.columns = ['color_label', 'value']
fig_k = px.pie(pie_k, values='value', names='color_label', height=400, width=400, color='color_label',
                 labels={"color_label": "Cluster", "value": "Número de Instancias"})

fig_k.show()

In [ ]:
from plotly.subplots import make_subplots
# Crear subplots
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "domain"}, {"type": "domain"}]], subplot_titles=("Gráfico tarta Agglomerative Clustering", "Gráfico de tarta KMeans"))

# Agregar gráfico polar
for trace in fig_a.data:
    fig.add_trace(trace, row=1, col=1)

# Agregar gráfico de tarta
for trace in fig_k.data:
    fig.add_trace(trace, row=1, col=2)

# Actualizar el layout para un título global
fig.update_layout(title_text="Resultados del Clustering", title_x=0.5, title_y=0.95, title_font_size=20)

# Mostrar la figura
fig.show()

In [ ]:
# Parámetros para DBSCAN
eps_values = np.linspace(0.1, 1.0, 10)
min_samples_values = range(1, 10)
results = []

# Calcular el tamaño mínimo del cluster en base al porcentaje
min_points_percentage = 0.01  # 5% del total
min_points = int(len(X) * min_points_percentage)

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan_model.fit_predict(X)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Excluir ruido
        
        # Verificar el tamaño mínimo de los clusters
        clusters = pd.Series([label for label in labels if label != -1])
        if all(clusters.value_counts() >= min_points):
            results.append((eps, min_samples, n_clusters))

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results, columns=['eps', 'min_samples', 'n_clusters'])
results_pivot = results_df.pivot_table(values='n_clusters', index='eps', columns='min_samples')

# Mostrar la tabla de resultados
print("Resultados del clustering con DBSCAN (número de clusters):")
print(results_pivot)

# Función para encontrar la combinación de parámetros más cercana al número objetivo de clusters
def find_best_dbscan_params(target_clusters, results_df):
    results_df['diff'] = abs(results_df['n_clusters'] - target_clusters)
    best_params = results_df.loc[results_df['diff'].idxmin()]
    return best_params['eps'], int(best_params['min_samples'])

# Especificar el número objetivo de clusters
target_clusters = 3

# Encontrar los mejores parámetros
best_eps, best_min_samples = find_best_dbscan_params(target_clusters, results_df)
print(f"Mejores parámetros para DBSCAN con objetivo de {target_clusters} clusters: eps={best_eps}, min_samples={best_min_samples}")

# Aplicar DBSCAN con los mejores parámetros encontrados
dbscan_model = DBSCAN(eps=best_eps, min_samples=best_min_samples)
y_dbscan = dbscan_model.fit_predict(X)

# Crear un DataFrame para clusters excluyendo el ruido
clusters_dbscan = pd.DataFrame(X, columns=X.columns)
clusters_dbscan['label'] = y_dbscan

# Excluir puntos de ruido
clusters_dbscan_valid = clusters_dbscan[clusters_dbscan['label'] != -1]

# Calcular las medias de las features para cada cluster
cluster_means = clusters_dbscan_valid.groupby('label').mean()

# Calcular la media global
global_mean = clusters_dbscan_valid.mean()

# Comparar las medias de cada cluster con la media global
influence = cluster_means - global_mean

influence.drop(columns=['label'], inplace=True)

# Mostrar la influencia de cada feature en cada cluster
print("Influencia de las features en cada cluster:")
print(influence)

# Visualización de la influencia de las features en cada cluster
plt.figure(figsize=(12, 8))
sns.heatmap(influence, annot=True, cmap='coolwarm')
plt.title('Influencia de las features en cada cluster')
plt.show()

In [ ]:

# Mostrar la influencia de cada feature en cada cluster
print("Influencia de las features en cada cluster:")
print(influence)

# Visualización de la influencia de las features en cada cluster
plt.figure(figsize=(12, 8))
sns.heatmap(influence, annot=True, cmap='coolwarm')
plt.title('Influencia de las features en cada cluster')
plt.show()

In [ ]:
# Calcular métricas de evaluación
silhouette_avg = silhouette_score(X, y_dbscan)
davies_bouldin_avg = davies_bouldin_score(X, y_dbscan)

# Imprimir métricas
print("Silhouette Score:", silhouette_avg)
print("Davies-Bouldin Index:", davies_bouldin_avg)
print("")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Suponiendo que X es un DataFrame de pandas y labels son las etiquetas de los clusters obtenidas de cada algoritmo

y_dbscan2  = y_dbscan[y_dbscan != -1]

Y = X.copy()

#quitar de Y los valores -1
Y['label'] = y_dbscan
Y = Y[Y['label'] != -1]

print(Y.shape)
print(y_dbscan2.shape)
print(X.shape)
print(y_dbscan.shape)

In [ ]:


# Visualización para DBSCAN
plt.figure(figsize=(10, 6))
plt.scatter(Y.iloc[:, 0], Y.iloc[:, 1], c=y_dbscan2, cmap='viridis')
plt.title('DBSCAN Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

In [ ]:


# Visualización para AgglomerativeClustering
plt.figure(figsize=(10, 6))
plt.scatter(Y.iloc[:, 0], X.iloc[:, 1], c=y_agg, cmap='viridis')
plt.title('Agglomerative Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

# Visualización para KMeans
plt.figure(figsize=(10, 6))
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y_kmeans, cmap='viridis')
plt.title('KMeans Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Suponiendo que X es un DataFrame de pandas y labels son las etiquetas de los clusters obtenidas de cada algoritmo

# Aplicar PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
Y_pca = pca.fit_transform(Y)

# Visualización para DBSCAN
plt.figure(figsize=(10, 6))
plt.scatter(Y_pca[:, 0], Y_pca[:, 1], c=y_dbscan2, cmap='viridis')
plt.title('DBSCAN Clustering (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Visualización para AgglomerativeClustering
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_agg, cmap='viridis')
plt.title('Agglomerative Clustering (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Visualización para KMeans
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_kmeans, cmap='viridis')
plt.title('KMeans Clustering (PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:

clusters_f = pd.DataFrame(X, columns=X.columns)

clusters_f['label'] = y_dbscan
clusters_f = clusters_f[clusters_f['label'] != -1]


color_map = {0: 'blue', 1: 'red', 2: 'green'}

# Visualización de distribución de clusters con Clustering Jerárquico
pie_k = clusters_f.groupby('label').size().reset_index()
pie_k.columns = ['label', 'value']
fig_k = px.pie(pie_k, values='value', names='label', color='label', height=400, width=400, color_discrete_map=color_map,
                 labels={"label": "Cluster", "value": "Número de Instancias"})

fig_k.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots

# Supongamos que X es tu conjunto de datos y ya ha sido definido

# Función para asignar colores en base al tamaño del cluster
def assign_colors_by_size(labels, color_order):
    cluster_sizes = pd.Series(labels).value_counts()
    sorted_clusters = cluster_sizes.index
    color_map = {sorted_clusters[i]: color_order[i] for i in range(len(sorted_clusters))}
    return labels.map(color_map)

# Parámetros para DBSCAN
eps_values = np.linspace(0.1, 1.0, 10)
min_samples_values = range(1, 10)
results = []

# Calcular el tamaño mínimo del cluster en base al porcentaje
min_points_percentage = 0.01  # 1% del total
min_points = int(len(X) * min_points_percentage)

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan_model.fit_predict(X)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Excluir ruido
        
        # Verificar el tamaño mínimo de los clusters
        clusters = pd.Series([label for label in labels if label != -1])
        if all(clusters.value_counts() >= min_points):
            results.append((eps, min_samples, n_clusters))

# Convertir los resultados a un DataFrame
results_df = pd.DataFrame(results, columns=['eps', 'min_samples', 'n_clusters'])
results_pivot = results_df.pivot_table(values='n_clusters', index='eps', columns='min_samples')

# Mostrar la tabla de resultados
print("Resultados del clustering con DBSCAN (número de clusters):")
print(results_pivot)

# Función para encontrar la combinación de parámetros más cercana al número objetivo de clusters
def find_best_dbscan_params(target_clusters, results_df):
    results_df['diff'] = abs(results_df['n_clusters'] - target_clusters)
    best_params = results_df.loc[results_df['diff'].idxmin()]
    return best_params['eps'], int(best_params['min_samples'])

# Especificar el número objetivo de clusters
target_clusters = 3

# Encontrar los mejores parámetros
best_eps, best_min_samples = find_best_dbscan_params(target_clusters, results_df)
print(f"Mejores parámetros para DBSCAN con objetivo de {target_clusters} clusters: eps={best_eps}, min_samples={best_min_samples}")

# Aplicar DBSCAN con los mejores parámetros encontrados
dbscan_model = DBSCAN(eps=best_eps, min_samples=best_min_samples)
y_dbscan = dbscan_model.fit_predict(X)

# Crear un DataFrame para clusters excluyendo el ruido
clusters_dbscan = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
clusters_dbscan['label'] = y_dbscan

# Excluir puntos de ruido
clusters_dbscan_valid = clusters_dbscan[clusters_dbscan['label'] != -1].copy()

# Asignar colores según el tamaño del cluster
color_order = ['blue', 'green', 'red']
clusters_dbscan_valid['color_label'] = assign_colors_by_size(clusters_dbscan_valid['label'], color_order)

# Calcular las medias de las features para cada cluster
cluster_means = clusters_dbscan_valid.groupby('color_label').mean()

# Calcular la media global
global_mean = clusters_dbscan_valid.mean(numeric_only=True)

# Comparar las medias de cada cluster con la media global
influence = cluster_means - global_mean

# Mostrar la influencia de cada feature en cada cluster
print("Influencia de las features en cada cluster:")
print(influence)

# Visualización de la influencia de las features en cada cluster
plt.figure(figsize=(12, 8))
sns.heatmap(influence, annot=True, cmap='coolwarm')
plt.title('Influencia de las features en cada cluster')
plt.show()

# Visualización de distribución de clusters con DBSCAN
pie_dbscan = clusters_dbscan_valid.groupby('color_label').size().reset_index()
pie_dbscan.columns = ['color_label', 'value']
fig_pie_dbscan = px.pie(pie_dbscan, values='value', names='color_label', color='color_label', 
                        color_discrete_map={'blue': 'blue', 'green': 'green', 'red': 'red'},
                        labels={"color_label": "Cluster", "value": "Número de Instancias"})

# Crear subplots para incluir el gráfico de tarta de DBSCAN
fig_dbscan = make_subplots(
    rows=1, cols=1,
    subplot_titles=("Gráfico de Tarta para Distribución de Clusters DBSCAN"),
    specs=[[{"type": "domain"}]]
)

# Agregar gráfico de tarta de DBSCAN
for trace in fig_pie_dbscan.data:
    fig_dbscan.add_trace(trace, row=1, col=1)

# Actualizar el layout para un título global
fig_dbscan.update_layout(title_text="Resultados del Clustering DBSCAN")

# Mostrar la figura
fig_dbscan.show()